# Preamble

In [88]:
import glob
import os
import pandas as pd
import numpy as np
import seaborn as sns
import wikitextparser as wtp
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize
from datetime import datetime
from ast import literal_eval

# Lists

In [424]:
# API variables
api_url = 'https://yugipedia.com/api.php'
sets_query_url = '?action=ask&query=[[Category:Set%20Card%20Lists]]|limit%3D5000|order%3Dasc&format=json'
lists_query_url = '?action=query&prop=revisions&rvprop=content&format=json&titles='

In [429]:
df = pd.read_json(f'{api_url}{sets_query_url}')
keys = list(df['query']['results'].keys())
print(len(keys))

3943


In [501]:
all_set_lists = pd.DataFrame(columns = ['Set','Card number','Name','Rarity','Print','Quantity'])

for i in range(40):
    first = i*100
    last = (i+1)*100
    print(len(keys[first:last]))
    titles = '|'.join(keys[first:last]).replace(' ','%20').encode('utf-8')
    df = pd.read_json(f'{api_url}{lists_query_url}{titles}')
    contents = df['query']['pages'].values()
    
    for content in contents:
        if 'revisions' in  content.keys():
            temp = content['revisions'][0]['*']
            parsed = wtp.parse(temp)
            title = content['title'].split('Lists:')[1]
            set_df = pd.DataFrame(columns = ['Set', 'Card number', 'Name', 'Rarity', 'Print', 'Quantity'])
            
            
            region = None
            rarity = None
            card_print = None
            qty = None
            desc = None
            opt = None
            list_df = None

            for template in parsed.templates:
                if template.name == 'Set list':
                    for argument in template.arguments:
                        if 'region=' in argument:
                            region = argument.string[argument.string.index('=')+1:]
                        elif 'rarities=' in argument:
                            rarity = argument.string[argument.string.index('=')+1:]
                        elif 'print=' in argument:
                            card_print = argument.string[argument.string.index('=')+1:]
                        elif 'qty=' in argument:
                            qty = argument.string[argument.string.index('=')+1:]
                        elif 'description=' in argument:
                            desc = argument.string[argument.string.index('=')+1:]
                        elif 'options=' in argument:
                            opt = argument.string[argument.string.index('=')+1:]
                        else:
                            set_list = argument.string[2:-1]
                            lines = set_list.split('\n')

                            list_df = pd.DataFrame([x.split(';') for x in lines])
                            list_df = list_df[~list_df[0].str.contains('!:')]
                            list_df.replace(r'^\s*$', None, regex = True, inplace = True)
                            list_df = list_df.applymap(lambda x: x.split('//')[0] if x is not None else x)
                            list_df = list_df.applymap(lambda x: x.strip() if x is not None else x)

                    if opt != 'noabbr':
                        set_df['Card number'] = list_df[0]
                        set_df['Name'] = list_df[1]
                    else: 
                        set_df['Name'] = list_df[0]

                    if len(list_df.columns)>2: # and rare in str
                        set_df['Rarity'] = list_df[2].apply(lambda x: x if x is not None else rarity)
                    else:
                        set_df['Rarity'] = rarity

                    if len(list_df.columns)>3 :
                        if card_print is not None: # and new/reprint in str
                            set_df['Print'] = list_df[3].apply(lambda x: x if x is not None else card_print)
                            if len(list_df.columns)>4 and qty is not None:
                                set_df['Quantity'] = list_df[4].apply(lambda x: x if x is not None else qty)
                        elif qty is not None:
                            set_df['Quantity'] = list_df[3].apply(lambda x: x if x is not None else qty)
                    
                    set_df['Set'] = title
                    all_set_lists = pd.concat([all_set_lists, set_df], ignore_index=True)

100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
43


In [502]:
all_set_lists

,Set,Card number,Name,Rarity,Print,Quantity
0,2013 Collectible Tins Wave 1 (TCG-EN),CT10-EN001,"Tidal, Dragon Ruler of Waterfalls",Secret Rare,NaN,NaN
1,2013 Collectible Tins Wave 1 (TCG-EN),CT10-EN005,Black Luster Soldier - Envoy of the Beginning,Super Rare,NaN,NaN
2,2013 Collectible Tins Wave 1 (TCG-EN),CT10-EN006,Ally of Justice Catastor,Super Rare,NaN,NaN
3,2013 Collectible Tins Wave 1 (TCG-EN),CT10-EN007,Superdreadnought Rail Cannon Gustav Max,Super Rare,NaN,NaN
4,2013 Collectible Tins Wave 1 (TCG-EN),CT10-EN002,"Blaster, Dragon Ruler of Infernos",Secret Rare,NaN,NaN
...,...,...,...,...,...,...
64708,King's Court (TCG-EN),KICO-EN061,The Wicked Avatar,R,Reprint,NaN
64709,King's Court (TCG-EN),KICO-EN062,The Wicked Eraser,R,Reprint,NaN
64710,King's Court (TCG-EN),KICO-EN063,Slifer the Sky Dragon,UR,Reprint,NaN
64711,King's Court (TCG-EN),KICO-EN064,Obelisk the Tormentor,UR,Reprint,NaN


In [503]:
all_set_lists[all_set_lists['Set']=='Legendary Decks II (TCG-PT)']

,Set,Card number,Name,Rarity,Print,Quantity
6607,Legendary Decks II (TCG-PT),LDK2-PTS01,Slifer the Sky Dragon,UR,NaN,NaN
6608,Legendary Decks II (TCG-PT),LDK2-PTS02,Obelisk the Tormentor,UR,NaN,NaN
6609,Legendary Decks II (TCG-PT),LDK2-PTS03,The Winged Dragon of Ra,UR,NaN,NaN
6610,Legendary Decks II (TCG-PT),LDK2-PTY01,The Legendary Exodia Incarnate,UR,NaN,NaN
6611,Legendary Decks II (TCG-PT),LDK2-PTY02,Ties of the Brethren,UR,NaN,NaN
...,...,...,...,...,...,...
6740,Legendary Decks II (TCG-PT),LDK2-PTT03,Token,UR,NaN,NaN
6741,Legendary Decks II (TCG-PT),None,None,C,NaN,NaN
6742,Legendary Decks II (TCG-PT),LDK2-PTS04,Dark Burning Attack,ScR,NaN,NaN
6743,Legendary Decks II (TCG-PT),LDK2-PTS05,Dark Burning Magic,ScR,NaN,NaN


In [508]:
all_set_lists.groupby('Card number', dropna=False).nunique()

,Set,Name,Rarity,Print,Quantity
Card number,,,,,
15AX-KR000,2,1,1,0,0
15AX-KRM01,1,1,1,0,0
15AX-KRM02,1,1,1,0,0
15AX-KRM03,1,1,1,0,0
15AX-KRM04,1,1,1,0,0
...,...,...,...,...,...
ZTIN-SP021,1,1,1,0,0
ZTIN-SPV01,1,1,1,0,0
ZTIN-SPV02,1,1,1,0,0


In [505]:
all_set_lists.groupby('Name', dropna=False).nunique()

,Set,Card number,Rarity,Print,Quantity
Name,,,,,
"""A"" Cell Breeding Device",5,5,2,0,0
"""A"" Cell Incubator",7,7,2,2,0
"""A"" Cell Recombination Device",4,4,1,0,0
"""A"" Cell Scatter Burst",5,5,2,0,0
"""Infernoble Arms - Durendal""",6,6,1,1,0
...,...,...,...,...,...
Zubaba Knight,22,22,7,1,1
Zubababancho Gagagacoat,2,2,1,1,0
"Zure, Knight of Dark World",11,11,2,0,1


In [506]:
all_set_lists.groupby('Rarity', dropna=False).nunique()

,Set,Card number,Name,Print,Quantity
Rarity,,,,,
,5,13,4,0,0
10000 Secret Rare,5,5,1,1,0
20th Secret Rare,8,16,18,0,0
20th Secret rare,1,2,2,0,0
C,392,16680,4460,2,3
...,...,...,...,...,...
Ultra rare,45,50,34,0,0
UtR,19,42,22,0,0
"UtR,C",4,4,2,0,1


In [507]:
all_set_lists.groupby('Print', dropna=False).nunique()

,Set,Card number,Name,Rarity,Quantity
Print,,,,,
European debut,1,2,2,1,0
New,129,3640,1835,44,3
New art,2,4,2,1,0
New artwork,5,16,2,3,0
Reprint,117,4742,2125,38,3
Reprint (New Art),1,1,1,1,0
NaN,1567,55170,9628,150,3


In [448]:
all_set_lists.groupby('Quantity', dropna=False).nunique()

,Card number,Name,Rarity,Print
Quantity,,,,
1,3446,967,12,2
2,377,133,4,2
3,22,10,2,2
NaN,59593,10366,166,6


In [454]:
all_set_lists[all_set_lists['Card number']=='!: header::Kaiba Deck']

,Card number,Name,Rarity,Print,Quantity
6653,!: header::Kaiba Deck,None,C,NaN,NaN
